In [1]:
%load_ext autoreload
%autoreload 2
import os, sys

In [2]:
import pandas as pd
import numpy as np
import plotly
from utils.analysis_utils import get_r_2
from preprocessing.data_processing import process_data, clean_data, cluster_rental_location

In [3]:
import pandas as pd
from datetime import datetime
import plotly.graph_objects as go
import plotly.express as px
from sklearn.cluster import KMeans
pd.options.plotting.backend = "plotly"

In [4]:
DATA_DIR = './data'
train = os.path.join(DATA_DIR, 'train.csv')
test = os.path.join(DATA_DIR, 'test.csv')

train_final = os.path.join(DATA_DIR, 'train_with_mrt_mall_school.csv')
test_final = os.path.join(DATA_DIR, 'test_with_mrt_mall_school.csv')
train_final_df, test_final_df = pd.read_csv(train_final), pd.read_csv(test_final)
train_df, test_df = pd.read_csv(train), pd.read_csv(test)

train_df_cleaned, test_df_cleaned = clean_data(train_df), clean_data(test_df)
train_df_cleaned, test_df_cleaned = process_data(train_df_cleaned), process_data(test_df_cleaned, mode='test')
train_df_cleaned_final, test_df_cleaned_final = clean_data(train_final_df), clean_data(train_final_df)
train_df_cleaned_final, test_df_cleaned_final = process_data(train_df_cleaned_final), process_data(test_df_cleaned_final, mode='test')


In [5]:
test_df_cleaned_final.columns

Index(['Unnamed: 0', 'list_id', 'rent_approval_date', 'town', 'block',
       'street_name', 'flat_type', 'flat_model', 'floor_area_sqm',
       'lease_commence_date', 'latitude', 'longitude', 'subzone',
       'planning_area', 'region', 'monthly_rent', 'near_mrt_count_0.3',
       'near_mrt_count_0.8', 'near_mrt_count_1.0', 'near_mrt_count_1.5',
       'near_mrt_count_2', 'nearest_code', 'nearest_mrt_name',
       'nearest_open_year', 'nearest_mrt_dist', 'near_mall_count_0.5',
       'near_mall_count_1.0', 'near_mall_count_1.5', 'near_mall_count_2',
       'nearest_mall_name', 'nearest_mall_dist', 'near_school_count_0.5',
       'near_school_count_1.0', 'near_school_count_1.5', 'near_school_count_2',
       'nearest_school_name', 'nearest_school_dist', 'type_model', 'date',
       'age', 'age_year', 'age_bin'],
      dtype='object')

## Check null data

In [6]:
train_df.isnull().any()

rent_approval_date     False
town                   False
block                  False
street_name            False
flat_type              False
flat_model             False
floor_area_sqm         False
furnished              False
lease_commence_date    False
latitude               False
longitude              False
elevation              False
subzone                False
planning_area          False
region                 False
monthly_rent           False
dtype: bool

In [7]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   rent_approval_date   60000 non-null  object 
 1   town                 60000 non-null  object 
 2   block                60000 non-null  object 
 3   street_name          60000 non-null  object 
 4   flat_type            60000 non-null  object 
 5   flat_model           60000 non-null  object 
 6   floor_area_sqm       60000 non-null  float64
 7   furnished            60000 non-null  object 
 8   lease_commence_date  60000 non-null  int64  
 9   latitude             60000 non-null  float64
 10  longitude            60000 non-null  float64
 11  elevation            60000 non-null  float64
 12  subzone              60000 non-null  object 
 13  planning_area        60000 non-null  object 
 14  region               60000 non-null  object 
 15  monthly_rent         60000 non-null 

In [8]:
train_df.furnished.unique()  # removed ?

array(['yes'], dtype=object)

## Clean the data

In [9]:
train_df_cleaned = clean_data(train_df)
train_df_cleaned = process_data(train_df_cleaned)
train_df_cleaned

,rent_approval_date,town,block,street_name,flat_type,flat_model,floor_area_sqm,lease_commence_date,latitude,longitude,...,region,monthly_rent,type_model,model_digit,type_digit,psqm,date,age,age_year,age_bin
0,2021-09,jurong east,257,jurong east street 24,3-room,new generation,67.0,1983,1.344518,103.738630,...,west region,1600,new generation 3-room,10,1,23.880597,1.630426e+09,14123,39.230556,NaN
1,2022-05,bedok,119,bedok north road,4-room,new generation,92.0,1978,1.330186,103.938717,...,east region,2250,new generation 4-room,10,2,24.456522,1.651334e+09,16191,44.975000,NaN
2,2022-10,toa payoh,157,lorong 1 toa payoh,3-room,improved,67.0,1971,1.332242,103.845643,...,central region,1900,improved 3-room,5,1,28.358209,1.664554e+09,18901,52.502778,NaN
3,2021-08,pasir ris,250,pasir ris street 21,executive,apartment,149.0,1993,1.370239,103.962894,...,east region,2850,apartment executive,3,4,19.127517,1.627747e+09,10439,28.997222,"[20.781, 30.0)"
4,2022-11,kallang/whampoa,34,whampoa west,3-room,improved,68.0,1972,1.320502,103.863341,...,central region,2100,improved 3-room,5,1,30.882353,1.667232e+09,18567,51.575000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,2021-09,ang mo kio,441,ang mo kio avenue 10,3-room,new generation,67.0,1979,1.366050,103.854168,...,north-east region,2200,new generation 3-room,10,1,32.835821,1.630426e+09,15584,43.288889,NaN
59996,2023-04,bukit merah,95a,henderson road,4-room,model a,83.0,2019,1.286493,103.821434,...,central region,4100,model a 4-room,7,2,49.397590,1.680278e+09,1551,4.308333,"[2.031, 9.531)"
59997,2022-06,tampines,862a,tampines street 83,5-room,improved,122.0,1988,1.355064,103.936507,...,east region,2250,improved 5-room,5,3,18.442623,1.654013e+09,12570,34.916667,NaN
59998,2023-01,bedok,67,bedok sth avenue 3,5-room,standard,123.0,1977,1.318974,103.944076,...,east region,4700,standard 5-room,15,3,38.211382,1.672502e+09,16801,46.669444,NaN


## Check column values

In [10]:
train_df_cleaned['flat_model'].unique(), len(train_df_cleaned['flat_model'].unique())
# train_df_cleaned[train_df_cleaned['flat_model'] == '2-room']

(array(['new generation', 'improved', 'apartment', 'premium apartment',
        'simplified', 'model a', 'standard', 'dbss', 'model a2',
        'maisonette', 'model a-maisonette', 'type s1', 'adjoined flat',
        'type s2', 'premium apartment loft', '2-room-model',
        'premium maisonette', 'terrace', '3gen'], dtype=object),
 19)

In [11]:
train_df_cleaned['flat_type'].unique(), len(train_df_cleaned['flat_type'].unique())

(array(['3-room', '4-room', 'executive', '5-room', '2-room'], dtype=object), 5)

In [12]:
train_df_cleaned['type_model'].unique(), len(train_df_cleaned['type_model'].unique())

(array(['new generation 3-room', 'new generation 4-room',
        'improved 3-room', 'apartment executive',
        'premium apartment executive', 'premium apartment 5-room',
        'simplified 4-room', 'model a 4-room', 'improved 5-room',
        'model a 3-room', 'standard 3-room', 'simplified 3-room',
        'dbss 4-room', 'model a2 4-room', 'improved 4-room',
        'premium apartment 4-room', 'model a 2-room',
        'maisonette executive', 'improved 2-room', 'standard 5-room',
        'model a 5-room', 'premium apartment 3-room', 'dbss 3-room',
        'dbss 5-room', 'model a-maisonette 5-room', 'type s1 4-room',
        'adjoined flat executive', 'type s2 5-room', 'standard 4-room',
        'adjoined flat 5-room', 'premium apartment loft 4-room',
        'standard 2-room', 'adjoined flat 4-room', '2-room-model 2-room',
        'premium maisonette executive', 'premium apartment 2-room',
        'terrace 3-room', 'terrace 4-room', '3gen 5-room'], dtype=object),
 39)

## Visualize the location cluster (First Glance)

In [13]:
data = train_df[['latitude', 'longitude']].to_dict(orient='list')
data.keys()

dict_keys(['latitude', 'longitude'])

In [14]:
def show_cluster_location(data, n_clusters):

    # df = pd.DataFrame(data)

    # # Number of clusters (change this based on your requirement)

    # # KMeans clustering
    # kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(df[['latitude', 'longitude']])
    # df['cluster'] = kmeans.labels_
    df = cluster_rental_location(train_df, n_clusters)[['latitude', 'longitude', 'cluster']]
    # Plotting on map using Plotly with Mapbox
    fig = px.scatter_mapbox(df,
                            lat='latitude',
                            lon='longitude',
                            color='cluster',
                            zoom=10,  # Adjust zoom level as needed
                            mapbox_style="open-street-map",  # Use open-street-map style
                            color_continuous_scale=px.colors.sequential.Plasma,
                            title="KMeans Clustering of rented house")

    fig.show()

In [15]:
show_cluster_location(data=data, n_clusters=6)

/Users/jiageliu/dev/lab/cs5228/.venv/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


variance by cluster

In [16]:
clustered_df = cluster_rental_location(train_df, 6)

/Users/jiageliu/dev/lab/cs5228/.venv/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [17]:
clustered_df['monthly_rent'].std()

714.9104679236704

In [18]:
clustered_df.groupby('cluster')['monthly_rent'].std()

cluster
0    648.552476
1    695.870302
2    705.253152
3    682.160297
4    840.744522
5    678.242028
Name: monthly_rent, dtype: float64

In [19]:
clustered_df.groupby('town')['monthly_rent'].std()

town
ang mo kio         642.344210
bedok              641.510567
bishan             739.559310
bukit batok        655.718657
bukit merah        830.599794
bukit panjang      683.366077
bukit timah        866.126153
central            966.797551
choa chu kang      661.580163
clementi           727.075709
geylang            708.573146
hougang            663.601764
jurong east        728.884426
jurong west        707.557602
kallang/whampoa    714.371025
marine parade      694.459191
pasir ris          689.280575
punggol            695.336359
queenstown         861.933204
sembawang          653.665074
sengkang           674.009024
serangoon          672.455401
tampines           691.754444
toa payoh          702.513826
woodlands          647.419316
yishun             642.572235
Name: monthly_rent, dtype: float64

In [20]:
clustered_df.groupby('region')['monthly_rent'].std()

region
central region       799.174469
east region          680.710535
north region         648.552476
north-east region    675.303569
west region          700.541628
Name: monthly_rent, dtype: float64

## Price for type

In [37]:
train_df_cleaned.groupby('flat_model')['monthly_rent'].std()

flat_model
2-room-model               696.905562
3gen                      1590.859307
adjoined flat             1138.741660
apartment                  721.599062
dbss                       712.651007
improved                   716.086106
maisonette                 765.423404
model a                    710.757682
model a-maisonette         828.086023
model a2                   615.333783
new generation             604.034005
premium apartment          704.108075
premium apartment loft     987.631848
premium maisonette        1747.212065
simplified                 646.724173
standard                   772.987472
terrace                   1441.353531
type s1                    971.717010
type s2                    912.045003
Name: monthly_rent, dtype: float64

In [38]:
train_df_cleaned.groupby('flat_type')['monthly_rent'].std()

flat_type
2-room       613.541815
3-room       560.914029
4-room       706.453729
5-room       729.186910
executive    739.069293
Name: monthly_rent, dtype: float64

In [39]:
train_df_cleaned.groupby('type_model')['monthly_rent'].std()

type_model
2-room-model 2-room               696.905562
3gen 5-room                      1590.859307
adjoined flat 4-room              922.632104
adjoined flat 5-room             1202.942820
adjoined flat executive          1123.387734
apartment executive               721.599062
dbss 3-room                       544.954509
dbss 4-room                       638.692633
dbss 5-room                       738.781916
improved 2-room                   499.009063
improved 3-room                   542.091337
improved 4-room                   662.964485
improved 5-room                   721.775298
maisonette executive              765.423404
model a 2-room                    648.130293
model a 3-room                    596.332804
model a 4-room                    709.708935
model a 5-room                    668.207096
model a-maisonette 5-room         828.086023
model a2 4-room                   615.333783
new generation 3-room             532.448167
new generation 4-room             653.983416

In [24]:
train_df_cleaned.groupby(['rent_approval_date', 'flat_model'])['psqm'].std().mean()

6.227164242562963

In [25]:
train_df_cleaned.groupby(['rent_approval_date', 'flat_type'])['psqm'].std().mean()

6.743394701378523

In [26]:
train_df_cleaned.groupby(['rent_approval_date', 'town'])['psqm'].std().mean()

6.705414946281903

In [41]:
train_df_cleaned.groupby(['rent_approval_date', 'subzone'])['psqm'].std().mean()

6.237375892708838

In [40]:
train_df_cleaned.groupby(['rent_approval_date', 'planning_area'])['psqm'].std().mean()

6.682063785041712

## Relationship with price

Apparently, monthly rent will be related to the floor_area_sqm, hence suppose we are able to predict price per square meter correctly, it's fairly easy to infer the price

In [42]:
train_df_cleaned_final['rent_town_perc'] = train_df_cleaned_final.groupby(['rent_approval_date', 'town'])[['monthly_rent']].rank(pct=True)
train_df_cleaned_final['area_town_perc'] = train_df_cleaned_final.groupby(['rent_approval_date', 'town'])[['floor_area_sqm']].rank(pct=True)
train_df_cleaned_final['rent_perc'] = train_df_cleaned_final.groupby(['rent_approval_date'])[['monthly_rent']].rank(pct=True)
train_df_cleaned_final['area_perc'] = train_df_cleaned_final.groupby(['rent_approval_date'])[['floor_area_sqm']].rank(pct=True)
train_df_cleaned_final['psqm_town_perc'] = train_df_cleaned_final.groupby(['rent_approval_date', 'town'])[['psqm']].rank(pct=True)
train_df_cleaned_final['psqm_perc'] = train_df_cleaned_final.groupby(['rent_approval_date'])[['psqm']].rank(pct=True)
train_df_cleaned_final.columns

Index(['Unnamed: 0', 'list_id', 'rent_approval_date', 'town', 'block',
       'street_name', 'flat_type', 'flat_model', 'floor_area_sqm',
       'lease_commence_date', 'latitude', 'longitude', 'subzone',
       'planning_area', 'region', 'monthly_rent', 'near_mrt_count_0.3',
       'near_mrt_count_0.8', 'near_mrt_count_1.0', 'near_mrt_count_1.5',
       'near_mrt_count_2', 'nearest_code', 'nearest_mrt_name',
       'nearest_open_year', 'nearest_mrt_dist', 'near_mall_count_0.5',
       'near_mall_count_1.0', 'near_mall_count_1.5', 'near_mall_count_2',
       'nearest_mall_name', 'nearest_mall_dist', 'near_school_count_0.5',
       'near_school_count_1.0', 'near_school_count_1.5', 'near_school_count_2',
       'nearest_school_name', 'nearest_school_dist', 'type_model',
       'model_digit', 'type_digit', 'psqm', 'date', 'age', 'age_year',
       'age_bin', 'rent_town_perc', 'area_town_perc', 'rent_perc', 'area_perc',
       'psqm_town_perc', 'psqm_perc'],
      dtype='object')

In [30]:
def plot_feature_relation(df, x, y, transform=lambda x: x):
    df = df.copy()
    df[x] = df[x].map(transform)
    return df.plot.scatter(x = x, y=y)

In [31]:
print('raw r^2 score = {}'.format(get_r_2(train_df_cleaned_final, 'floor_area_sqm', 'monthly_rent')))
print('date level percentile = {}'.format(get_r_2(train_df_cleaned_final, 'floor_area_sqm', 'rent_perc')))
print('town level percentile = {}'.format(get_r_2(train_df_cleaned_final, 'floor_area_sqm', 'rent_town_perc')))

raw r^2 score = 0.09392125317005673
date level percentile = 0.16507916013954715
town level percentile = 0.1630367932521174


In [32]:
print('raw r^2 score = {}'.format(get_r_2(train_df_cleaned_final, 'age', 'monthly_rent')))
print('date level percentile = {}'.format(get_r_2(train_df_cleaned_final, 'age', 'rent_perc')))
print('town level percentile = {}'.format(get_r_2(train_df_cleaned_final, 'age', 'rent_town_perc')))

raw r^2 score = 0.037166012723239716
date level percentile = 0.06461484554046448
town level percentile = 0.0537374209644198


In [43]:
print('raw r^2 score = {}'.format(get_r_2(train_df_cleaned_final, 'lease_commence_date', 'monthly_rent')))
print('date level percentile = {}'.format(get_r_2(train_df_cleaned_final, 'lease_commence_date', 'rent_perc')))
print('town level percentile = {}'.format(get_r_2(train_df_cleaned_final, 'lease_commence_date', 'rent_town_perc')))

raw r^2 score = 0.051060457787462044
date level percentile = 0.06464953376608129
town level percentile = 0.05379336266434642


In [35]:
print('raw r^2 score = {}'.format(get_r_2(train_df_cleaned_final, 'near_mrt_count', 'monthly_rent')))
print('date level percentile = {}'.format(get_r_2(train_df_cleaned_final, 'near_mrt_count', 'rent_perc')))
print('town level percentile = {}'.format(get_r_2(train_df_cleaned_final, 'near_mrt_count', 'rent_town_perc')))

KeyError: 'near_mrt_count'

In [ ]:
print('raw r^2 score = {}'.format(get_r_2(train_df_cleaned_final, 'nearest_mrt_dist', 'monthly_rent')))
print('date level percentile = {}'.format(get_r_2(train_df_cleaned_final, 'nearest_mrt_dist', 'rent_perc')))
print('town level percentile = {}'.format(get_r_2(train_df_cleaned_final, 'nearest_mrt_dist', 'rent_town_perc')))

raw r^2 score = 0.004496792421567752
date level percentile = 0.006807107723963979
town level percentile = 0.008164421582299797


In [36]:
print('raw r^2 score = {}'.format(get_r_2(train_df_cleaned_final, 'near_mrt_count', 'psqm')))
print('date level percentile = {}'.format(get_r_2(train_df_cleaned_final, 'near_mrt_count', 'psqm_perc')))
print('town level percentile = {}'.format(get_r_2(train_df_cleaned_final, 'near_mrt_count', 'psqm_town_perc')))

KeyError: 'near_mrt_count'